In [ ]:
from langgraph.graph.message import add_messages, BaseMessage
from langchain_openai import ChatOpenAI
from dotenv import dotenv_values
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

config = dotenv_values(".env")

In [ ]:
def add_messages_with_limit(left, right, limit=4):
    merged = add_messages(left, right)
    return merged[-limit:]

In [ ]:
@tool
def get_latest_news(place:str)->str:
    """
    Returns latest news based on the place
    Args:
        place (str): Name of a place

    Returns:
        str: Current News
    """
    print("I am invoked: get_latest_news")
    if place == "Asia":
        return "There is a war"
    else:
        return "There is peace"

@tool
def get_latest_weather(place:str)->str:
    """
    Returns latest weather based on the place
    Args:
        place (str): Name of a place


    Returns:
        str: Current Weather
    """    
    print("I am invoked: get_latest_weather")
    if place == "Kolkata":
        return "It is Rainy"
    else:
        return "It is Sunny"

In [ ]:
class State(TypedDict):
    messages: Annotated[list[BaseMessage], lambda l, r: add_messages_with_limit(l, r, limit=4)]

In [ ]:
llm = ChatOpenAI(api_key=config["OPEN_AI_API_KEY"], model="gpt-4o-mini", temperature=0.1)

In [ ]:
graph_builder = StateGraph(State)

tools = [get_latest_news, get_latest_weather]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print("Exception occured {}".format(e))

In [ ]:
config = {"configurable": {"thread_id": "1"}}

def stream_graph_updates(user_input: str):
    response = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config)
    print("Assistant:", response["messages"][-1].content)
    print(len(response["messages"]))
            
while True:
    user_input = input("User: ")
    print("User:", user_input)
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Assistant: ", "Goodbye!")
        break
    stream_graph_updates(user_input)